# Event Study (Dynamic DiD) with `pymc` models

This notebook demonstrates how to use CausalPy's `EventStudy` class to estimate **dynamic treatment effects** over event time. This is also known as a "dynamic {term}`difference in differences`" analysis. The {term}`event study` is a powerful tool for:
1. Examining **pre-treatment trends** (placebo checks for {term}`parallel trends assumption`)
2. Estimating how **treatment effects evolve over time** after treatment
3. Visualizing the full **time path of causal effects**

## Background: What is an Event Study?

An event study analyzes panel data where some units receive treatment at a specific time. Unlike standard difference-in-differences which estimates a single average treatment effect, event studies estimate **separate coefficients for each time period relative to treatment**.

The key concept is **event time** (or relative time):

$$E_{it} = t - G_i$$

where $t$ is the calendar time and $G_i$ is the treatment time for unit $i$.

The model estimates {cite:t}`sun2021estimating`:

$$Y_{it} = \alpha_i + \lambda_t + \sum_{k \neq k_0} \beta_k \cdot \mathbf{1}\{E_{it} = k\} + \varepsilon_{it}$$

where:
- $\alpha_i$ are unit fixed effects
- $\lambda_t$ are time fixed effects
- $\beta_k$ are the dynamic treatment effects at event time $k$
- $k_0$ is the reference (omitted) period, typically $k=-1$

**Interpretation:**
- $\beta_k$ for $k < 0$ (pre-treatment): Should be near zero if parallel trends hold
- $\beta_k$ for $k \geq 0$ (post-treatment): Measure the causal effect at each period after treatment

:::{warning}
This implementation uses a standard two-way fixed effects (TWFE) estimator, which requires **simultaneous treatment timing** - all treated units must receive treatment at the same time. Staggered adoption designs (where different units are treated at different times) can produce biased estimates when treatment effects vary across cohorts {footcite:t}`sun2021estimating`.
:::


In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import seaborn as sns

import causalpy as cp
from causalpy.data.simulate_data import generate_event_study_data

In [ ]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'
seed = 42
# Set arviz style to override seaborn's default
az.style.use("arviz-darkgrid")

## Generate Simulated Data

We'll create synthetic panel data with:
- 30 units (half treated, half control)
- 20 time periods
- Treatment occurring at time 10
- Known treatment effects: zero pre-treatment, gradually increasing post-treatment


In [ ]:
# Define known treatment effects for simulation
# Pre-treatment: no effect (parallel trends)
# Post-treatment: effect increases over time
true_effects = {
    -5: 0.0,
    -4: 0.0,
    -3: 0.0,
    -2: 0.0,
    -1: 0.0,  # Pre-treatment
    0: 0.5,
    1: 0.7,
    2: 0.9,
    3: 1.0,
    4: 1.0,
    5: 1.0,  # Post-treatment
}

df = generate_event_study_data(
    n_units=30,
    n_time=20,
    treatment_time=10,
    treated_fraction=0.5,
    event_window=(-5, 5),
    treatment_effects=true_effects,
    unit_fe_sigma=1.0,
    time_fe_sigma=0.3,
    noise_sigma=0.2,
    seed=seed,
)

print(f"Data shape: {df.shape}")
df.head(10)

Let's visualize the data to understand its structure:


In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sns.lineplot(
    data=df,
    x="time",
    y="y",
    hue="treated",
    units="unit",
    estimator=None,
    alpha=0.5,
    ax=ax,
)
ax.axvline(x=10, color="red", linestyle="--", linewidth=2, label="Treatment time")
ax.set(
    xlabel="Time", ylabel="Outcome (y)", title="Panel Data: Treated vs Control Units"
)
ax.legend(loc="upper left")
plt.show()

## Run the Event Study Analysis

Now we use CausalPy's `EventStudy` class to estimate the dynamic treatment effects.

The `formula` parameter uses patsy syntax to specify the model structure:
- `y ~ C(unit) + C(time)` specifies the outcome variable `y` with unit fixed effects ($\alpha_i$) and time fixed effects ($\lambda_t$)
- `C(column)` indicates a categorical variable that should be converted to dummy variables

**What about the $\beta_k$ coefficients?** These are the event-time dummies - the key parameters we want to estimate. They capture the treatment effect at each period *relative to treatment* (i.e., at each event time $k$). The class automatically constructs these based on:
- The `event_window` parameter (e.g., `(-5, 5)` means $k \in \{-5, -4, ..., 0, ..., 5\}$)
- The `reference_event_time` parameter (e.g., `-1` is omitted as the baseline)

So the formula specifies the "structural" part of the model ($\alpha_i + \lambda_t$), while the event-time dummies ($\sum_{k \neq k_0} \beta_k \cdot \mathbf{1}\{E_{it} = k\}$) are added automatically.

:::{note}
The `random_seed` keyword argument for the PyMC sampler is not necessary. We use it here so that the results are reproducible.
:::

In [ ]:
result = cp.EventStudy(
    df,
    formula="y ~ C(unit) + C(time)",  # Outcome with unit and time fixed effects
    unit_col="unit",
    time_col="time",
    treat_time_col="treat_time",
    event_window=(-5, 5),
    reference_event_time=-1,  # One period before treatment as reference
    model=cp.pymc_models.LinearRegression(sample_kwargs={"random_seed": seed}),
)

## Visualize the Results

The event study plot shows the estimated treatment effects ($\beta_k$) at each event time, with credible intervals. This is the key diagnostic plot for event studies.


In [ ]:
fig, ax = result.plot(figsize=(8, 5))

**Interpreting the Plot:**

1. **Pre-treatment periods** ($k < 0$, blue shaded): The coefficients should be close to zero. This is a key test of the parallel trends assumption. If we see significant pre-trends, our causal estimates may be biased.

2. **Reference period** ($k = -1$, orange square): This is fixed at zero by construction. All other coefficients are interpreted relative to this period.

3. **Post-treatment periods** ($k \geq 0$): These show how the treatment effect evolves over time. In our simulated data, we see the effect starts at about 0.5 and increases to around 1.0.

4. **Credible intervals**: The error bars show 94% highest density intervals. When these don't include zero for post-treatment periods, we have strong evidence of a treatment effect.

:::{tip}
The plot can be customized with optional parameters:
- `figsize=(width, height)` to change the figure size
- `hdi_prob=0.89` to change the credible interval (e.g., 89% instead of 94%)

Example: `result.plot(figsize=(12, 8), hdi_prob=0.89)`
:::


## Summary Statistics


In [ ]:
result.summary()

We can also get the event-time coefficients as a DataFrame for further analysis:


In [ ]:
summary_df = result.get_event_time_summary()
summary_df

## Compare Estimated vs True Effects

Since we simulated the data with known treatment effects, we can compare our estimates to the true values:


In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

# Plot estimated effects
event_times = summary_df["event_time"].values
estimated_means = summary_df["mean"].values
lower = summary_df["hdi_3%"].values
upper = summary_df["hdi_97%"].values

ax.errorbar(
    event_times,
    estimated_means,
    yerr=[estimated_means - lower, upper - estimated_means],
    fmt="o",
    capsize=4,
    capthick=2,
    markersize=8,
    color="C0",
    label="Estimated (with 94% HDI)",
)

# Plot true effects (relative to k=-1 reference)
# Since k=-1 is our reference, we need to subtract true_effects[-1] from all
true_effects_relative = {k: v - true_effects[-1] for k, v in true_effects.items()}
true_k = list(true_effects_relative.keys())
true_beta = list(true_effects_relative.values())
ax.scatter(
    true_k, true_beta, marker="x", s=100, color="red", zorder=5, label="True effect"
)

ax.axhline(y=0, color="gray", linestyle="--", alpha=0.7)
ax.axvline(x=0, color="gray", linestyle=":", alpha=0.7)
ax.set_xlabel("Event Time (k)")
ax.set_ylabel(r"$\beta_k$ (Treatment Effect)")
ax.set_title("Estimated vs True Treatment Effects")
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

The estimated effects closely track the true effects, demonstrating that the event study correctly recovers the dynamic treatment effects.


## Pre-Trend Analysis

A key assumption in event studies is **parallel trends**: without treatment, treated and control units would have followed similar outcome trajectories. We can assess this by examining the pre-treatment coefficients.

If parallel trends hold, pre-treatment coefficients ($\beta_k$ for $k < 0$) should be close to zero:


In [ ]:
# Extract pre-treatment coefficients
pre_treatment = summary_df[summary_df["event_time"] < 0]
pre_treatment = pre_treatment[~pre_treatment["is_reference"]]

print("Pre-treatment coefficient estimates:")
print(pre_treatment[["event_time", "mean", "hdi_3%", "hdi_97%"]].to_string(index=False))

# Check if 94% HDIs include zero
pre_includes_zero = (
    (pre_treatment["hdi_3%"] <= 0) & (pre_treatment["hdi_97%"] >= 0)
).all()
print(f"\nAll pre-treatment 94% HDIs include zero: {pre_includes_zero}")

Since all pre-treatment coefficient credible intervals include zero, we have no evidence against the parallel trends assumption. This gives us confidence in our post-treatment causal estimates.


## Posterior Analysis

We can use ArviZ to examine the posterior distributions of specific event-time coefficients:


In [ ]:
# Plot posterior for event time k=0 (immediate treatment effect)
if 0 in result.event_time_coeffs and hasattr(result.event_time_coeffs[0], "values"):
    ax = az.plot_posterior(result.event_time_coeffs[0].values.flatten(), ref_val=0)
    ax.set_title(r"Posterior of $\beta_0$ (Immediate treatment effect)")

In [ ]:
# Plot posterior for event time k=5 (long-run treatment effect)
if 5 in result.event_time_coeffs and hasattr(result.event_time_coeffs[5], "values"):
    ax = az.plot_posterior(result.event_time_coeffs[5].values.flatten(), ref_val=0)
    ax.set_title(r"Posterior of $\beta_5$ (Long-run treatment effect)")

## Key Takeaways

1. **Event studies** provide richer information than standard DiD by estimating treatment effects at each event time.

2. **Pre-trend analysis** is crucial: coefficients for $k < 0$ should be near zero to support the parallel trends assumption.

3. **Dynamic effects** can reveal how treatment impacts evolve over time—whether effects are immediate, gradual, or temporary.

4. **The reference period** ($k_0$, typically -1) is normalized to zero. All coefficients are interpreted relative to this period.

5. **Bayesian estimation** provides full posterior distributions for each coefficient, enabling probabilistic statements about effect sizes.


## References

:::{bibliography}
:filter: docname in docnames
:::
